# Metrics Calculation Module Demo

This notebook demonstrates the comprehensive metrics calculation capabilities implemented in Task 2.1.

**Features:**
- Analytical metrics calculation based on paper formulas
- Empirical metrics from simulation results
- Comparison between analytical and empirical values
- Energy efficiency analysis
- Latency and throughput metrics
- Batch analysis with confidence intervals

**Date:** February 10, 2026

In [ ]:
# Setup path for imports
import sys
from pathlib import Path

# Add parent directory to path
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# Import simulator and metrics modules
from src.simulator import Simulator, SimulationConfig, BatchSimulator
from src.power_model import PowerModel, PowerProfile
from src.metrics import (
    MetricsCalculator,
    AnalyticalMetrics,
    analyze_batch_results
)

print("✓ Imports successful!")
print(f"Project root: {project_root}")

## 1. Analytical Metrics Calculation

First, let's compute analytical metrics based on the paper formulas without running any simulation.

In [ ]:
# System parameters
n = 20          # Number of nodes
q = 0.05        # Transmission probability
lambda_rate = 0.01  # Arrival rate
tw = 5          # Wake-up time
ts = 10         # Idle timer

# Compute analytical metrics
analytical = MetricsCalculator.compute_analytical_metrics(
    n=n,
    q=q,
    lambda_rate=lambda_rate,
    tw=tw,
    ts=ts,
    has_sleep=True
)

print("📐 Analytical Metrics (from paper formulas)")
print("=" * 60)
print(f"Success probability (p): {analytical.success_probability:.6f}")
print(f"Service rate (μ): {analytical.service_rate:.6f}")
print(f"Mean queue length (¯L): {analytical.mean_queue_length:.4f}")
print(f"Mean delay (¯T): {analytical.mean_delay:.4f} slots")
print(f"Stability condition (λ < μ): {analytical.stability_condition}")

# Compare with optimal q
optimal_q = MetricsCalculator.compute_optimal_q(n)
p_optimal = MetricsCalculator.compute_analytical_success_probability(n, optimal_q)
print(f"\nOptimal q = 1/n: {optimal_q:.4f}")
print(f"Success prob at optimal q: {p_optimal:.6f}")
print(f"Current success prob: {analytical.success_probability:.6f}")
print(f"Ratio (current/optimal): {analytical.success_probability/p_optimal:.2%}")

## 2. Run Simulation and Compute Empirical Metrics

Now let's run a simulation and compute empirical metrics from the results.

In [ ]:
# Create simulation configuration
config = SimulationConfig(
    n_nodes=20,
    arrival_rate=0.01,
    transmission_prob=0.05,
    idle_timer=10,
    wakeup_time=5,
    initial_energy=10000,
    power_rates=PowerModel.get_profile(PowerProfile.GENERIC_LOW),
    max_slots=50000,
    seed=42
)

# Run simulation with history tracking
print("Running simulation...")
sim = Simulator(config)
result = sim.run_simulation(track_history=True, verbose=True)

print("\n✓ Simulation complete!")

## 3. Comprehensive Metrics Analysis

Compute all metrics including analytical comparison.

In [ ]:
# Compute comprehensive metrics
metrics = MetricsCalculator.compute_comprehensive_metrics(
    result,
    include_analytical=True
)

# Print formatted summary
MetricsCalculator.print_metrics_summary(metrics, verbose=True)

## 4. Detailed Metric Categories

Let's examine each metric category in detail.

In [ ]:
# Energy efficiency metrics
energy_metrics = MetricsCalculator.compute_energy_efficiency_metrics(result)

print("🔋 Energy Efficiency Metrics")
print("=" * 60)
for key, value in energy_metrics.items():
    print(f"{key:30s}: {value:.6f}")

In [ ]:
# Latency metrics
latency_metrics = MetricsCalculator.compute_latency_metrics(result)

print("⏱️  Latency Metrics")
print("=" * 60)
for key, value in latency_metrics.items():
    print(f"{key:30s}: {value:.4f}")

In [ ]:
# Network performance metrics
network_metrics = MetricsCalculator.compute_network_performance_metrics(result)

print("🌐 Network Performance Metrics")
print("=" * 60)
for key, value in network_metrics.items():
    if isinstance(value, float):
        print(f"{key:30s}: {value:.6f}")
    else:
        print(f"{key:30s}: {value}")

## 5. Visualize Energy Breakdown

In [ ]:
# Energy breakdown pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Energy by state
energy_fractions = result.energy_fractions_by_state
labels = [s.capitalize() for s in energy_fractions.keys()]
values = list(energy_fractions.values())
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']

ax1.pie(values, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Energy Consumption by State', fontsize=14, fontweight='bold')

# Time in state
state_fractions = result.state_fractions
labels2 = [s.capitalize() for s in state_fractions.keys()]
values2 = list(state_fractions.values())

ax2.pie(values2, labels=labels2, autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Time Fraction by State', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 Key Observation:")
print(f"Sleep mode accounts for {energy_fractions.get('sleep', 0)*100:.1f}% of energy")
print(f"but {state_fractions.get('sleep', 0)*100:.1f}% of time")

## 6. Queue Length Time Series Analysis

In [ ]:
# Queue length statistics
queue_stats = MetricsCalculator.compute_queue_length_statistics(
    result.queue_length_history
)

print("📈 Queue Length Statistics")
print("=" * 60)
for key, value in queue_stats.items():
    print(f"{key:15s}: {value:.4f}")

# Plot queue length over time
fig, ax = plt.subplots(figsize=(14, 5))

slots = np.arange(len(result.queue_length_history))
ax.plot(slots, result.queue_length_history, linewidth=0.8, alpha=0.7)
ax.axhline(queue_stats['mean'], color='r', linestyle='--', label=f"Mean: {queue_stats['mean']:.2f}")
ax.axhline(queue_stats['p95'], color='orange', linestyle='--', label=f"95th: {queue_stats['p95']:.2f}")

ax.set_xlabel('Slot Number', fontsize=12)
ax.set_ylabel('Average Queue Length', fontsize=12)
ax.set_title('Queue Length Evolution Over Time', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Analytical vs Empirical Comparison

Compare simulation results with analytical predictions.

In [ ]:
# Get analytical and comparison metrics
analytical_metrics = metrics['analytical']
comparison = metrics['comparison']

# Create comparison table
import pandas as pd

comparison_data = {
    'Metric': ['Success Probability', 'Service Rate', 'Mean Delay'],
    'Empirical': [
        f"{result.empirical_success_prob:.6f}",
        f"{result.empirical_service_rate:.6f}",
        f"{result.mean_delay:.2f}"
    ],
    'Analytical': [
        f"{analytical_metrics['success_probability']:.6f}",
        f"{analytical_metrics['service_rate']:.6f}",
        f"{analytical_metrics['mean_delay']:.2f}"
    ],
    'Relative Error': [
        f"{comparison['success_prob_error']:.2%}",
        f"{comparison['service_rate_error']:.2%}",
        f"{comparison['delay_error']:.2%}"
    ]
}

df = pd.DataFrame(comparison_data)
print("\n📊 Empirical vs Analytical Comparison")
print("=" * 80)
print(df.to_string(index=False))

if comparison['warnings']:
    print("\n⚠️  Warnings:")
    for warning in comparison['warnings']:
        print(f"  - {warning}")

## 8. Batch Analysis with Multiple Replications

Run multiple replications and compute confidence intervals.

In [ ]:
# Run batch simulations
print("Running 10 replications...")
batch_config = SimulationConfig(
    n_nodes=20,
    arrival_rate=0.01,
    transmission_prob=0.05,
    idle_timer=10,
    wakeup_time=5,
    initial_energy=5000,
    power_rates=PowerModel.get_profile(PowerProfile.GENERIC_LOW),
    max_slots=20000,
    seed=None
)

batch_sim = BatchSimulator(batch_config)
batch_results = batch_sim.run_replications(n_replications=10, verbose=True)

# Analyze batch results
aggregated = analyze_batch_results(batch_results)

print("\n📊 Aggregated Results (Mean ± Std)")
print("=" * 60)
for metric_name, (mean, std) in aggregated.items():
    if metric_name == 'lifetime_years':
        print(f"{metric_name:25s}: {BatchSimulator.format_lifetime(mean, std)}")
    elif 'delay' in metric_name:
        print(f"{metric_name:25s}: {mean:.2f} ± {std:.2f} slots")
    else:
        print(f"{metric_name:25s}: {mean:.6f} ± {std:.6f}")

## 9. Parameter Sweep: Effect of Transmission Probability q

Sweep q values and observe impact on metrics.

In [ ]:
# Parameter sweep for q
q_values = [0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
n_reps = 5

print(f"Running parameter sweep over {len(q_values)} q values with {n_reps} replications each...")

sweep_config = SimulationConfig(
    n_nodes=20,
    arrival_rate=0.01,
    transmission_prob=0.05,  # Will be overridden
    idle_timer=10,
    wakeup_time=5,
    initial_energy=3000,
    power_rates=PowerModel.get_profile(PowerProfile.GENERIC_LOW),
    max_slots=15000,
    seed=None
)

batch_sim = BatchSimulator(sweep_config)
sweep_results = batch_sim.parameter_sweep(
    param_name='transmission_prob',
    param_values=q_values,
    n_replications=n_reps,
    verbose=True
)

print("\n✓ Parameter sweep complete!")

In [ ]:
# Analyze sweep results
sweep_analysis = {}
for q_val, results in sweep_results.items():
    agg = analyze_batch_results(results)
    sweep_analysis[q_val] = agg

# Extract metrics for plotting
q_array = np.array(q_values)
mean_delays = [sweep_analysis[q]['mean_delay'][0] for q in q_values]
delay_stds = [sweep_analysis[q]['mean_delay'][1] for q in q_values]
lifetimes = [sweep_analysis[q]['lifetime_years'][0] for q in q_values]
lifetime_stds = [sweep_analysis[q]['lifetime_years'][1] for q in q_values]
throughputs = [sweep_analysis[q]['throughput'][0] for q in q_values]

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Mean delay vs q
ax = axes[0, 0]
ax.errorbar(q_array, mean_delays, yerr=delay_stds, marker='o', capsize=5, linewidth=2)
ax.set_xlabel('Transmission Probability (q)', fontsize=11)
ax.set_ylabel('Mean Delay (slots)', fontsize=11)
ax.set_title('Mean Delay vs Transmission Probability', fontweight='bold')
ax.grid(True, alpha=0.3)

# Lifetime vs q
ax = axes[0, 1]
# Convert to hours for better readability
lifetimes_hours = [lt * 365.25 * 24 for lt in lifetimes]
lifetime_stds_hours = [std * 365.25 * 24 for std in lifetime_stds]
ax.errorbar(q_array, lifetimes_hours, yerr=lifetime_stds_hours, marker='s', capsize=5, linewidth=2, color='green')
ax.set_xlabel('Transmission Probability (q)', fontsize=11)
ax.set_ylabel('Mean Lifetime (hours)', fontsize=11)
ax.set_title('Lifetime vs Transmission Probability', fontweight='bold')
ax.grid(True, alpha=0.3)

# Throughput vs q
ax = axes[1, 0]
ax.plot(q_array, throughputs, marker='^', linewidth=2, markersize=8, color='purple')
ax.set_xlabel('Transmission Probability (q)', fontsize=11)
ax.set_ylabel('Throughput (packets/slot)', fontsize=11)
ax.set_title('Throughput vs Transmission Probability', fontweight='bold')
ax.grid(True, alpha=0.3)

# Lifetime-Delay tradeoff
ax = axes[1, 1]
ax.scatter(mean_delays, lifetimes_hours, s=100, c=q_array, cmap='viridis', edgecolor='black')
for i, q_val in enumerate(q_values):
    ax.annotate(f'q={q_val}', (mean_delays[i], lifetimes_hours[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)
ax.set_xlabel('Mean Delay (slots)', fontsize=11)
ax.set_ylabel('Mean Lifetime (hours)', fontsize=11)
ax.set_title('Lifetime-Delay Tradeoff', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Findings:")
print(f"- Minimum delay: {min(mean_delays):.2f} slots at q={q_values[np.argmin(mean_delays)]}")
print(f"- Maximum lifetime: {max(lifetimes_hours):.2f} hours at q={q_values[np.argmax(lifetimes_hours)]}")
print(f"- Maximum throughput: {max(throughputs):.6f} at q={q_values[np.argmax(throughputs)]}")

## 10. Summary and Conclusions

This notebook demonstrated the comprehensive metrics calculation module (Task 2.1), including:

### ✅ Implemented Features:

1. **Analytical Metrics**: Computed directly from paper formulas
   - Success probability: p = q(1-q)^(n-1)
   - Service rate: μ with and without sleep
   - Mean delay and queue length from M/M/1 theory

2. **Empirical Metrics**: Extracted from simulation results
   - Lifetime estimation (slots and years)
   - Delay statistics (mean, 95th, 99th percentiles)
   - Energy efficiency metrics
   - Network performance (throughput, collisions, delivery ratio)

3. **Comparison Framework**: Validates simulation against theory
   - Relative error calculation
   - Stability condition checking
   - Warning system for invalid comparisons

4. **Batch Analysis**: Statistical aggregation across replications
   - Mean and standard deviation computation
   - Confidence interval estimation

5. **Visualization Support**: Time series and tradeoff analysis
   - Queue length evolution
   - Energy breakdown
   - Parameter sweep plots

### 🎯 Task 2.1 Status: ✅ COMPLETE

All required metrics from the PRD have been implemented and validated.